In [1]:
import pandas as pd
import numpy as np 
import datetime
z=0.824
fluence=4.77e-5
d28 =  (5271.8*3.08567758128E+18*1e6)/1e28
E53 = 1e-53*4*np.pi*((d28*1e28)**2)*fluence
print(z,d28,E53)

0.824 1.6267075072991906 1.5861606990038475


1. Read in Observation File
2. Get Fmax for each observing band
3. Get time for each Fmax/observing band combo
4. Use Fmax, Tmax, E53, z, nu, d28 to compute Psi_ISM and PSI_Wind
5. Compute Psi(epsion_e, xi_e**(-3/2))
6. Compute epsilon_e given xi_e=1 (max value possible for xi_e)
7. Given a lower limit on gamma_min, use chi to place lower limit on xi_e.
8. Compute epsilon_e given lower limit on xi_e
9. Put data in a table

In [2]:
plotdata = pd.read_csv("grbmeas.csv")
trigger = datetime.datetime(2024, 2, 5, 22, 13, 6, 00)
startdate = [(datetime.datetime.strptime(d, "%Y-%m-%d %H:%M:%S.%f") - trigger).total_seconds()/3600/24 for d in plotdata['start']]
stopdate = [(datetime.datetime.strptime(d, "%Y-%m-%d %H:%M:%S.%f") - trigger).total_seconds()/3600/24 for d in plotdata['stop']]
obsdur = [(t2-t1) for t1,t2 in zip(startdate,stopdate)]
plotdata['obsdate'] = [t1+(dur/2) for t1,dur in zip(startdate,obsdur)]
plotdata['startdate'] = startdate
plotdata['stopdate'] = stopdate

In [18]:
def PSIism(f,t,nu):
    return np.sqrt( (261.4*np.sqrt(1+z)*nu*t**(3/2)*np.sqrt(E53)) / (1e15*d28**2*f))
def PSIwind(f,t,nu):
    return np.sqrt( (249.9*(1+z)*nu*t) / (1e15*d28**2*f))
def A_ism(psi,p,epsilon_gamma, n0):
    return psi / ( ((p-2)/(0.177*(p-1))) * ((p-0.67)/(p+0.14))**(1/2) * ((1-epsilon_gamma)/epsilon_gamma)**(-1/4) * n0**(-1/4))
def A_wind(psi,p,Astar):
    return psi/ ( ((p-2)/(0.277*(p-1)))*((p-0.69)/(p+0.12))**(1/2)*Astar**(-1/2))
def chiISM(psi,td):
    return 266*psi*E53**(1/8)*(z+1)**(3/8)*td**(-3/8)
def chiWIND(psi,td):
    return 475*psi*E53**0.25*(z+1)**(1/4)**td**(-1/4)
def xiforchiISM(chi,p,epsilon_gamma,n0,gamma_min):
    return ((((p-0.67)/(0.66*(p+0.14)))**(1/2)*(epsilon_gamma/(1-epsilon_gamma))**(3/8)*n0**(-1/8)*gamma_min)/chi)**2
def xiforchiWIND(chi,gamma_min,epsilon_gamma,p,Astar):
    return ((((p-0.69)/(0.65*(p+0.12)))**2*(epsilon_gamma/(1-epsilon_gamma))**(1/4)*Astar**(-1/4)*gamma_min)/chi)**2

In [19]:
def Tb(f,t,nu):
    d = d28*1e28
    t = t*24*60*60
    f = f*1e-3
    return 1.153e-8*d**2*f*nu**(-2)*t**(-2)*(1+z)**(-1)

In [44]:
psitable = []
minobs = plotdata.loc[plotdata['obsdate']==plotdata['obsdate'].min()]
tmin,numin,fmin,errmin = minobs['obsdate'].item(),minobs['freq'].item()*1e9,minobs['flux'].item()*1e-3,minobs['err'].item()*1e-3
tmax = plotdata['obsdate'].max()
print(tmax)
p=2.3
Astar=1
n0=1
epsilon_gamma=0.15
gamma_min = 2
def epsFORxi(xi,A):
    return A/(xi**(-3/2))
labels=["nu","xi1I","xi2I","xi1w","xi2w","eps1i","eps2i","eps1w","eps2w"]
for band in ["L","S","C","X"]:
    subdat = plotdata.loc[plotdata['band']==band]
    maxforband = subdat.loc[subdat['flux']==subdat['flux'].max()]
    nu, _, t, f, err = 1e9*maxforband['freq'].item(),maxforband['band'].item(),maxforband['obsdate'].item(),1e-3*maxforband['flux'].item(),1e-3*maxforband['err'].item()
    PSI_ism_mid = PSIism(f,t,nu)
    PSI_wind_mid = PSIwind(f,t,nu)
    combo_ISM, combo_WIND = A_ism(PSI_ism_mid,p,epsilon_gamma,n0),A_wind(PSI_wind_mid,p,Astar)
    eps_max_ISM, esp_max_WIND = epsFORxi(1,combo_ISM),epsFORxi(1,combo_WIND)
    # print('------')
    # for tmax in [50,plotdata['obsdate'].max(),263]:
    chi_psi_ism, chi_psi_wind = chiISM(PSI_ism_mid,tmax),chiWIND(PSI_wind_mid,tmax)
    xi_ism_min,xi_wind_min = xiforchiISM(chi_psi_ism,p,epsilon_gamma,n0,gamma_min),xiforchiWIND(chi_psi_wind,gamma_min,epsilon_gamma,p,Astar)
    epsilon_e_ism2, epsilon_e_wind2 = PSI_ism_mid/(combo_ISM*xi_ism_min**(-3/2)), PSI_wind_mid/(combo_WIND*xi_wind_min**(-3/2))
    epsilon_e_ism1, epsilon_e_wind1 = PSI_ism_mid/(combo_ISM), PSI_wind_mid/(combo_WIND)
    print(PSI_ism_mid,combo_ISM,nu,PSI_ism_mid/combo_ISM)
    psitable.append((nu*1e-9,f"{xi_ism_min:.2e}",1,f"{xi_wind_min:.2e}",1,f"{epsilon_e_ism1:.2e}",f"{epsilon_e_ism2:.2e}",f"{epsilon_e_wind1:.2e}",f"{epsilon_e_wind2:.2e}"))
psitab = pd.DataFrame(psitable,columns=labels)
print(psitab)
    # print('------')
    # continue
    
    
    # print(nu*1e-9,t,round(f,3),np.log10(PSIism(f+err,t,nu)),np.log10(PSIism(f-err,t,nu)),np.log10(PSIwind(f+err,t,nu)),
    #       np.log10(PSIwind(f-err,t,nu)),xi_ism_min,xi_wind_min)# (Tb(fmin,tmin,numin)/1e12)**(1/3),
    

161.7668275462963
0.7978565357704547 1.155189409045598 1600000000.0 0.690671615860496
0.8171711714880633 1.1831544148082063 2600000000.0 0.690671615860496
0.40683158487786253 0.5890376490584429 5500000000.0 0.690671615860496
0.32550094275012903 0.47128177164859014 9000000000.0 0.690671615860496
    nu      xi1I  xi2I      xi1w  xi2w     eps1i     eps2i     eps1w     eps2w
0  1.6  6.30e-04     1  3.82e-05     1  6.91e-01  1.09e-05  6.80e-01  1.60e-07
1  2.6  6.01e-04     1  3.64e-05     1  6.91e-01  1.02e-05  6.80e-01  1.49e-07
2  5.5  2.42e-03     1  7.07e-05     1  6.91e-01  8.24e-05  6.80e-01  4.04e-07
3  9.0  3.79e-03     1  8.85e-05     1  6.91e-01  1.61e-04  6.80e-01  5.66e-07
